# Content

A recommender system, or a recommendation system, is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item. They are primarily used in commercial applications. (source - Wikipedia)

Mainly three types of recommendation systems in machine learning based on filtering are used to suggest product and services to the consumers.

i. Content Filtering

ii. Collaborative Filtering

iii. Hybrid Filtering

1. Content Filtering:
In this algorithm, we try finding items look alike. Once we have item look like matrix,we can easily recommend alike items to a customer, who has purchased any item from the store.

2. Collaborative Filtering:
Here, we try to search for look alike customers and offer products based on what his/her lookalike has chosen.This algorithm is very effective but takes a lot of time and resources.

3. Hybrid Filtering (Content Filtering + Collaborative Filtering):
Both Content Filtering & Collaborative Filtering is used for the purpose. you-tube uses this algorithm for their strong recommendation system.



# Business Problem

We are provided with a dataset containing a set of data of Movie details, Movie Ratings and User information.
we are required to perform necessary analysis to build a recommendation system to recommend movies based on a given movie.

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

## Let us Import and Explore the Dataset

In [2]:
# Importing the data from csv file

rating = pd.read_csv("[5]Ratings.csv")
movie_d = pd.read_csv("[3]Movie details.csv", encoding='latin-1')
user_info = pd.read_csv("[4]user level info.csv")

In [3]:
# Showing the top of the data

rating.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
# Showing the top of the data

movie_d.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [5]:
# Showing the top of the data

user_info.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
# Shape of the data - ratings

rating.shape
print("No of records:", rating.shape[0])
print("No of Attributes:", rating.shape[1])

No of records: 100000
No of Attributes: 4


In [7]:
# Shape of the data - movie details

movie_d.shape
print("No of records:", movie_d.shape[0])
print("No of Attributes:", movie_d.shape[1])

No of records: 1682
No of Attributes: 24


In [8]:
# Shape of the data - user information

user_info.shape
print("No of records:", user_info.shape[0])
print("No of Attributes:", user_info.shape[1])

No of records: 943
No of Attributes: 5


In [9]:
# Renaming the columns of movie details data to remove unwanted space

movie_d.columns = ['movieid', 'movietitle', 'releasedate', 'videoreleasedate',
       'IMDbURL', 'unknown', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [10]:
# Renaming the columns of user information data to remove unwanted space

rating.columns = ['userid', 'movieid', 'rating', 'timestamp']

In [11]:
# Merging the data of movie details and ratings

movie_rating = pd.merge(rating,movie_d,on='movieid')

In [12]:
movie_rating

,userid,movieid,rating,timestamp,movietitle,releasedate,videoreleasedate,IMDbURL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,Kolya (1996),24-Jan-97,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,63,242,3,875747190,Kolya (1996),24-Jan-97,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,226,242,5,883888671,Kolya (1996),24-Jan-97,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,154,242,3,879138235,Kolya (1996),24-Jan-97,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,306,242,5,876503793,Kolya (1996),24-Jan-97,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,840,1674,4,891211682,Mamma Roma (1962),1-Jan-62,NaN,http://us.imdb.com/M/title-exact?Mamma%20Roma%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99996,655,1640,3,888474646,"Eighth Day, The (1996)",1-Nov-96,NaN,"http://us.imdb.com/Title?Huiti%E8me+jour,+Le+(...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99997,655,1637,3,888984255,Girls Town (1996),23-Aug-96,NaN,http://us.imdb.com/M/title-exact?Girls%20Town%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1...",2-Feb-96,NaN,http://us.imdb.com/M/title-exact?Saimt%20el%20...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [13]:
# Statistical measurements of the data

movie_rating.describe().T

,count,mean,std,min,25%,50%,75%,max
userid,100000.0,4.624848e+02,2.666144e+02,1.0,254.0,447.0,682.0,943.0
movieid,100000.0,4.255301e+02,3.307984e+02,1.0,175.0,322.0,631.0,1682.0
rating,100000.0,3.529860e+00,1.125674e+00,1.0,3.0,4.0,4.0,5.0
timestamp,100000.0,8.835289e+08,5.343856e+06,874724710.0,879448709.5,882826944.0,888259984.0,893286638.0
videoreleasedate,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unknown,100000.0,1.000000e-04,9.999550e-03,0.0,0.0,0.0,0.0,1.0
Action,100000.0,2.558900e-01,4.363625e-01,0.0,0.0,0.0,1.0,1.0
Adventure,100000.0,1.375300e-01,3.444077e-01,0.0,0.0,0.0,0.0,1.0
Animation,100000.0,3.605000e-02,1.864155e-01,0.0,0.0,0.0,0.0,1.0
Children,100000.0,7.182000e-02,2.581909e-01,0.0,0.0,0.0,0.0,1.0


In [14]:
# Let us now count the movies and the users

print('No. of Movies',movie_rating['movieid'].unique().shape[0])
print('No. of Users',movie_rating['userid'].unique().shape[0])

No. of Movies 1682
No. of Users 943


In [15]:
#Now showing the mean of ratings of the movie with respect to movie titles using groupby function and making it into a dataframe

movie_stats = pd.DataFrame(movie_rating.groupby(by='movietitle')['rating'].mean())

In [16]:
movie_stats.head(20)

,rating
movietitle,
'Til There Was You (1997),2.333333
1-900 (1994),2.600000
101 Dalmatians (1996),2.908257
12 Angry Men (1957),4.344000
187 (1997),3.024390
2 Days in the Valley (1996),3.225806
"20,000 Leagues Under the Sea (1954)",3.500000
2001: A Space Odyssey (1968),3.969112
3 Ninjas: High Noon At Mega Mountain (1998),1.000000


In [17]:
# Sorting the movie ratings in decreasing manner

movie_stats.sort_values(by='rating', ascending=False)

,rating
movietitle,
They Made Me a Criminal (1939),5.0
Marlene Dietrich: Shadow and Light (1996),5.0
"Saint of Fort Washington, The (1993)",5.0
Someone Else's America (1995),5.0
Star Kid (1997),5.0
...,...
"Eye of Vichy, The (Oeil de Vichy, L') (1993)",1.0
King of New York (1990),1.0
Touki Bouki (Journey of the Hyena) (1973),1.0


In [18]:
# Grouping the movies in accordance with the count of ratings received by a particular movie

movie_rating.groupby(by=['movietitle'])['userid'].count()

movietitle
'Til There Was You (1997)                  9
1-900 (1994)                               5
101 Dalmatians (1996)                    109
12 Angry Men (1957)                      125
187 (1997)                                41
                                        ... 
Young Guns II (1990)                      44
Young Poisoner's Handbook, The (1995)     41
Zeus and Roxanne (1997)                    6
unknown                                    9
Á köldum klaka (Cold Fever) (1994)         1
Name: userid, Length: 1664, dtype: int64

In [19]:
# Making a dataframe for the count of ratings recieved by a movie

movie_stats['count_rating'] = pd.DataFrame(movie_rating.groupby(by=['movietitle'])['userid'].count())

In [20]:
movie_stats

,rating,count_rating
movietitle,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41
...,...,...
Young Guns II (1990),2.772727,44
"Young Poisoner's Handbook, The (1995)",3.341463,41
Zeus and Roxanne (1997),2.166667,6


In [21]:
movie_stats.sort_values(by='count_rating', ascending=False)

,rating,count_rating
movietitle,,
Star Wars (1977),4.358491,583
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485
...,...,...
"Great Day in Harlem, A (1994)",5.000000,1
"Other Voices, Other Rooms (1997)",3.000000,1
Good Morning (1971),1.000000,1


In [22]:
user_movie_rating = movie_rating.pivot_table(index='userid',columns='movietitle',values='rating')

In [23]:
user_movie_rating

movietitle  'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
userid                                                                       
1                                 NaN           NaN                    2.0   
2                                 NaN           NaN                    NaN   
3                                 NaN           NaN                    NaN   
4                                 NaN           NaN                    NaN   
5                                 NaN           NaN                    2.0   
...                               ...           ...                    ...   
939                               NaN           NaN                    NaN   
940                               NaN           NaN                    NaN   
941                               NaN           NaN                    NaN   
942                               NaN           NaN                    NaN   
943                               NaN           NaN                    NaN   

movietitle  12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
userid                                                                     
1                           5.0         NaN                          NaN   
2                           NaN         NaN                          NaN   
3                           NaN         2.0                          NaN   
4                           NaN         NaN                          NaN   
5                           NaN         NaN                          NaN   
...                         ...         ...                          ...   
939                         NaN         NaN                          NaN   
940                         NaN         NaN                          NaN   
941                         NaN         NaN                          NaN   
942                         NaN         NaN                          NaN   
943                         NaN         NaN                          2.0   

movietitle  20,000 Leagues Under the Sea (1954)  2001: A Space Odyssey (1968)  \
userid                                                                          
1                                           3.0                           4.0   
2                                           NaN                           NaN   
3                                           NaN                           NaN   
4                                           NaN                           NaN   
5                                           NaN                           4.0   
...                                         ...                           ...   
939                                         NaN                           NaN   
940                                         NaN                           NaN   
941                                         NaN                           NaN   
942                                         NaN                           3.0   
943                                         NaN                           NaN   

movietitle  3 Ninjas: High Noon At Mega Mountain (1998)  39 Steps, The (1935)  \
userid                                                                          
1                                                   NaN                   NaN   
2                                                   1.0                   NaN   
3                                                   NaN                   NaN   
4                                                   NaN                   NaN   
5                                                   NaN                   NaN   
...                                                 ...                   ...   
939                                                 NaN                   NaN   
940                                                 NaN                   NaN   
941                                                 NaN                   NaN   
942                                                

In [24]:
# to check which users have rated a specific movie
user_movie_rating['101 Dalmatians (1996)']    

userid
1      2.0
2      NaN
3      NaN
4      NaN
5      2.0
      ... 
939    NaN
940    NaN
941    NaN
942    NaN
943    NaN
Name: 101 Dalmatians (1996), Length: 943, dtype: float64

In [25]:
# to check which movies are rated by a specific user
user_movie_rating.iloc[0,:]

movietitle
'Til There Was You (1997)                NaN
1-900 (1994)                             NaN
101 Dalmatians (1996)                    2.0
12 Angry Men (1957)                      5.0
187 (1997)                               NaN
                                        ... 
Young Guns II (1990)                     NaN
Young Poisoner's Handbook, The (1995)    NaN
Zeus and Roxanne (1997)                  NaN
unknown                                  4.0
Á köldum klaka (Cold Fever) (1994)       NaN
Name: 1, Length: 1664, dtype: float64

## Correlation Similarity

In [26]:
correlation = user_movie_rating.corrwith(user_movie_rating['101 Dalmatians (1996)'])

In [27]:
correlation.sort_values(ascending=False).head(20)

movietitle
Hard Rain (1998)                1.0
Winter Guest, The (1997)        1.0
Fatal Instinct (1993)           1.0
Faithful (1996)                 1.0
Trial by Jury (1994)            1.0
April Fool's Day (1986)         1.0
House Party 3 (1994)            1.0
Grateful Dead (1995)            1.0
Tie That Binds, The (1995)      1.0
Frisk (1995)                    1.0
In the Line of Duty 2 (1987)    1.0
Lightning Jack (1994)           1.0
Bye Bye, Love (1995)            1.0
101 Dalmatians (1996)           1.0
Loch Ness (1995)                1.0
Stranger, The (1994)            1.0
Boys (1996)                     1.0
Scarlet Letter, The (1926)      1.0
Palmetto (1998)                 1.0
SubUrbia (1997)                 1.0
dtype: float64

### Top Movies in the dataset

In [29]:
movie_stats['Factor'] = movie_stats['rating']*movie_stats['count_rating']

In [30]:
# The Top 20 movies in the dataset

movie_stats.sort_values(by='Factor',ascending=False).head(20)

,rating,count_rating,Factor
movietitle,,,
Star Wars (1977),4.358491,583,2541.0
Fargo (1996),4.155512,508,2111.0
Return of the Jedi (1983),4.007890,507,2032.0
Contact (1997),3.803536,509,1936.0
Raiders of the Lost Ark (1981),4.252381,420,1786.0
"Godfather, The (1972)",4.283293,413,1769.0
"English Patient, The (1996)",3.656965,481,1759.0
Toy Story (1995),3.878319,452,1753.0
"Silence of the Lambs, The (1991)",4.289744,390,1673.0


## Cosine Similarity

Let us take two movies randomly to see the cosine similarity between the two

In [31]:
user_movie_rating['101 Dalmatians (1996)'].fillna(0)

userid
1      2.0
2      0.0
3      0.0
4      0.0
5      2.0
      ... 
939    0.0
940    0.0
941    0.0
942    0.0
943    0.0
Name: 101 Dalmatians (1996), Length: 943, dtype: float64

In [32]:
user_movie_rating['Star Wars (1977)'].fillna(0)

userid
1      5.0
2      5.0
3      0.0
4      5.0
5      4.0
      ... 
939    0.0
940    4.0
941    0.0
942    5.0
943    4.0
Name: Star Wars (1977), Length: 943, dtype: float64

In [33]:
# Let us look at the cosine similarity between the two

In [34]:
# Importing the necessary libraries

from scipy import spatial
from numpy import dot
from numpy.linalg import norm

In [35]:
# Converting the movies into list and filling the null value with 0 since we need the dot product of the two

List1 = list(user_movie_rating['101 Dalmatians (1996)'].fillna(0))
List2 = list(user_movie_rating['Star Wars (1977)'].fillna(0))

In [37]:
result = dot(List1,List2)/(norm(List1)*norm(List2))

In [38]:
result

0.33995263376305596

As we see, the result ,i.e., the cosine similarity between the movies 101 Dalmatians (1996) and Star Wars (1977) comes out to be 0.33995 which indicates that the two movies are dis-similar from each other.

### Recommendation Function

Let us now make a function for determining the recommendation where we can input any movie and check the recommended movies based on it.

In [39]:
def recommendation(movie):
    simi = []
    mov = []
    List1 = list(user_movie_rating[movie].fillna(0))
    for i in movie_d['movietitle']:
        List2 = list(user_movie_rating[i].fillna(0))
        cos = dot(List1,List2)/(norm(List1)*norm(List2))
        simi.append(cos)
        mov.append(i)
    recommended_data = pd.DataFrame({'Movie':pd.Series(mov),'Similarity':pd.Series(simi)})
    return recommended_data.sort_values(by='Similarity',ascending=False).iloc[1:,:]

In [41]:
recommendation('101 Dalmatians (1996)').head(20)

,Movie,Similarity
408,Jack (1996),0.447883
117,Twister (1996),0.441882
150,Willy Wonka and the Chocolate Factory (1971),0.423295
120,Independence Day (ID4) (1996),0.420642
0,Toy Story (1995),0.409386
755,Father of the Bride Part II (1995),0.403524
595,"Hunchback of Notre Dame, The (1996)",0.403382
70,"Lion King, The (1994)",0.403309
392,Mrs. Doubtfire (1993),0.399968
464,"Jungle Book, The (1994)",0.397600


This shows all the movies recommended based on the movie 101 Dalmatians (1996) with the similarity rate.